## tagcrawler

In [ ]:
from bs4 import BeautifulSoup
import time
import re
from Requester import Requester
class TagCrawler(Requester) :
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.\
                94 Safari/537.36'
               }
    def __init__(self , url) :          #設定TagCrawler建構子
        self.url = url                  #啟動url
                        
    def allsubCraw(self):
        soup = super(TagCrawler,self).req(self.url)
        root ={}
        for url in soup.select('.more a'):  
            root[url.text] = url['href']
        return  root 
    def leafCraw(self,bigUrl):
        soup = super(TagCrawler,self).req(bigUrl)
        leafUrl = []
        rg = 'leaf'
        for url in soup.select('.title'):            
            isLeaf=re.search(rg,url['href'])
            try:
                isLeaf.group()              #y拍的小項網址裡面會有leaf這個字，利用這點濾掉不是小項的網址
                leafUrl.append(url['href'])
            except:
                   continue
        return leafUrl
    def startCraw(self):
        tc = TagCrawler('https://tw.bid.yahoo.com/tw/0-all.html?.r=1465414707')    
        root = tc.allsubCraw()
        leafUrl=[]    
        for url in root:
            leafUrl.extend(tc.leafCraw(root[url]))
        print len(leafUrl)
    # if main 裡面是單獨測試碼，直接執行可以得到測試結果                          
# if __name__ == '__main__':
#     tc = TagCrawler('https://tw.bid.yahoo.com/tw/0-all.html?.r=1465414707')    
#     root = tc.allsubCraw()
#     leafUrl=[]    
#     for url in root:
#         leafUrl.extend(tc.leafCraw(root[url]))
#     print len(leafUrl)

## requester

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
class Requester(object):
    headers = {
        'Accept':'*/*' ,
        'Accept-Encoding':'gzip, deflate, sdch, br' ,
        'Accept-Language':'zh-TW,zh;q=0.8,ja;q=0.6,en-US;q=0.4,en;q=0.2,zh-CN;q=0.2' ,
        'Connection':'keep-alive',    
        'Host':'tw.bid.yahoo.com' ,
        
        'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.84 Safari/537.36',
        'X-PJAX':'true',
        'X-Requested-With':'XMLHttpRequest' 
    }
    def req(self,url):
        connection = True
        soup=''
        rs = requests.session()
        while connection == True:           
            try:                
                res = rs.get(url,headers=self.headers)                
                soup = BeautifulSoup(res.text)                
                connection = False
                
                return soup
            except:
                print 'connection error , sleep 1 min'                          
                time.sleep(60)
                
                

## category

In [ ]:
from Requester import Requester
import requests
from bs4 import BeautifulSoup
import time
import re

class CategoryCrawler(Requester):
    next =''
    domainUrl = 'https://tw.bid.yahoo.com/tw/'
    def __init__(self , url) :          #設定CategoryCrawler建構子
        self.url = url
        self.hasNext = True                      
    def firstCraw(self):                #爬第一次，目的是取回下一頁連結
        soup = super(CategoryCrawler,self).req(self.url)
        nextpage = soup.select('.next-page a')[0]['href']
        self.targetTag=(soup.select('#srp_bc span')[-1].text).encode('utf-8')    #這裡python內(unicode環境)->編碼為utf-8(str)
        print self.targetTag  
        instantBuy=soup.select('.yui3-u.hasbuyp a')[0]['href']                              #抓一下這個最小類別的名稱
        return instantBuy
        
    def urlGather(self,url):                                #把一頁的網址收集起來 回傳一個List
        soup = super(CategoryCrawler,self).req(self.url)
        urlList =list()
        for title in soup.select('.srp-pdtitle'):
            print title.text
            #print title.text
            urlList.append(title.select('a')[0]['href'])
            
        try:   
            self.next =soup.select('.next-page.yui3-u a')[0]['href']               #如果現在爬取的這頁是最後一頁
        except:
            self.hasNext = False             
        return urlList     
    def contentCraw(self,url):
        soup = super(CategoryCrawler,self).req(self.url)
        content =''
        title = soup.select('.title')[0].text
        price = soup.select('.number')[0].text
        salCount = self.numberCleaner(soup.select('.has-sold')[0].text)
        remark = self.numberCleaner(soup.select('.remark')[0].text)                
        sallerName = soup.select('.seller-name a')[0].text   
        question = self.numberCleaner(soup.select('.total-item-count ')[0].text)         
        qicUrl = soup.select('.main-image')[0]['src']
        result =title+'_|'+url+'_|'+salCount+'_|'+price+'_|'+sallerName+'_|'+remark+'_|'+question+'_|'+qicUrl+'_|'+content
                #標題, 商品網址, 已賣數量, 價格, 賣家名稱 , 賣家評價 , 問與答數量 , 圖片網址 , 內文
        return result
             
    def getThisCate(self,actualPage,index):
        #組合網址，先拆出pg 
        address=actualPage.split('&')
        #塞入第n頁        
        address[-5]= 'pg={}'.format(index)
        address[-7]= 'aoffset={}'.format((index-1)*60)
        #放回actualPage
        actualPage = '&'.join(address)
        
        return actualPage 
    def numberCleaner(self,st):
        match = re.search(r'[\d,]+',st)
        num = match.group().encode('utf-8')
        return  num            
    # if main 裡面是單獨測試碼，直接執行可以得到測試結果         
# if __name__ == '__main__':
#     #在這裡需要輸入從標籤進來以後的第一個頁面   
#     cc = CategoryCrawler('https://tw.bid.yahoo.com/tw/2092109949-category-leaf.html?.r=1465376779?hpp=23336_cat_category')
#     #爬第一次  
#                                                                                   # ^^^^^^^^^^^^ 似乎只需要改這裡就能換標籤
#     actualPage = cc.firstCraw()     
#     urlList = []
#     index =1
#     #如果物件的hasNext=True 則執行迴圈
     
#     while cc.hasNext:       
#     # pg改成 .format形式在塞回成網址        
#         #actualPage=cc.getThisCate(actualPage,index)
        
        
#         print actualPage    
#         urlList.extend(cc.urlGather(actualPage))
#         actualPage = cc.domainUrl+cc.next
     
          
#         time.sleep(3)
#      #如果沒有下一頁 停止程式 po 出搜集到的url個數
#         if cc.hasNext ==False:
#             index =1            
#             print '沒有下一頁 {}完成'.format(cc.targetTag)
#             print len(urlList)  
#         else:
#                 index+=1

## maincrawler

In [ ]:
from TagCrawler import * 
from  CategoryCrawler import *
import time
import codecs

if __name__ == '__main__':
    tc = TagCrawler('https://tw.bid.yahoo.com/tw/0-all.html?.r=1465414707')
    rootDir = tc.allsubCraw()
    leafUrl=[]    
    for url in rootDir:
        leafUrl.extend(tc.leafCraw(rootDir[url]))
        print url ,'完成'
        time.sleep(1)
    print len(leafUrl)
    urlList = []
    
    for url in leafUrl[25:26]:
        index =1
        urlList = [] 
        cc = CategoryCrawler(url)
        actualPage = cc.firstCraw()        
            
        while cc.hasNext:       
            # pg改成 .format形式在塞回成網址             
            actualPage=cc.getThisCate(actualPage,index)
            page = cc.domainUrl+actualPage
            urlList.extend(cc.urlGather(page))
            print page
            time.sleep(1)
             #如果沒有下一頁 停止程式 po 出搜集到的url個數
            if cc.hasNext ==False:
                index =1            
                print '沒有下一頁 {}完成'.format(cc.targetTag)
                print len(urlList)  
            else:
                    index+=1
        f = codecs.open('e:\result\{}.txt'.format(cc.targetTag),'w')        
        for i in urlList :
            result=cc.contentCraw(i)
            f.write(result+'\n')        
            
